# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

In [1]:
from fireplace import cards, exceptions, utils
from fireplace.player import Player
from fireplace.game import Game
from fireplace.deck import Deck
from fireplace.utils import get_script_definition, random_draft

In [2]:
from hearthstone.enums import PlayState, Step, Mulligan, State, CardClass, Race, CardSet, CardType

In [3]:
import re
import string
import random

In [4]:
cards.db.initialize()

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 26667 cards


In [5]:
GREEN = "\033[92m"
RED = "\033[91m"
ENDC = "\033[0m"
PREFIXES = {
    GREEN: "Implemented",
    RED: "Not implemented",
}
implemented_cards = []
unimplemented_cards = []

In [6]:
SOLVED_KEYWORDS = [
    "Windfury", "Charge", "Divine Shield", "Taunt", "Stealth", "Poisonous",
    r"Can't be targeted by spells or Hero Powers\.",
    r"Can't attack\.",
    "Destroy any minion damaged by this minion.",
    r"Your Hero Power deals \d+ extra damage.",
    r"Spell Damage \+\d+",
    r"Overload: \(\d+\)",
]

In [7]:
DUMMY_CARDS = (
    "PlaceholderCard",  # Placeholder Card
    "CS2_022e",  # Polymorph
    "EX1_246e",  # Hexxed
    "EX1_345t",  # Shadow of Nothing
    "GAME_006",  # NOOOOOOOOOOOO
    "LOEA04_27",  # Animated Statue
    "Mekka4e",  # Transformed
    "NEW1_025e",  # Bolstered (Unused)
    "TU4c_005",  # Hidden Gnome
    "TU4c_007",  # Mukla's Big Brother

    # Dynamic buffs set by their parent
    "CS2_236e",  # Divine Spirit
    "EX1_304e",  # Consume (Void Terror)
    "LOE_030e",  # Hollow (Unused)
    "NEW1_018e",  # Treasure Crazed (Bloodsail Raider)
)

Por algun motivo separo unas cartas que llamo IMPLEMENTADAS de unas cartas que llamo NO_IMPLEMENTADAS

In [8]:
for id in sorted(cards.db):
    card = cards.db[id]
    ret = card.description
    ret = re.sub("<i>.+</i>", "", ret)
    ret = re.sub("(<b>|</b>)", "", ret)
    ret = re.sub("(" + "|".join(SOLVED_KEYWORDS) + ")", "", ret)
    ret = re.sub("<[^>]*>", "", ret)
    exclude_chars = string.punctuation + string.whitespace
    ret = "".join([ch for ch in ret if ch not in exclude_chars])
    description = ret
    implemented = False

    if not description:
        # Minions without card text or with basic abilities are implemented
        implemented = True
    elif card.card_set == CardSet.CREDITS:
        implemented = True

    if id in DUMMY_CARDS:
        implemented = True

    carddef = get_script_definition(id)
    if carddef:
        implemented = True

    color = GREEN if implemented else RED
    name = color + "%s: %s" % (PREFIXES[color], card.name) + ENDC

    if implemented:
        implemented_cards.append(card.id)
    else:
        unimplemented_cards.append(card.id)

IMPLEMENTED_CARDS = len(implemented_cards)
UNIMPLEMENTED_CARDS = len(unimplemented_cards)

print("IMPLEMENTED CARDS: "+str(IMPLEMENTED_CARDS))
print("UNIMPLEMENTED CARDS: "+str(UNIMPLEMENTED_CARDS))

IMPLEMENTED CARDS: 8042
UNIMPLEMENTED CARDS: 18625


In [9]:
implemented_cards[0:5]

['AIBot_DemonHunterTrainee_001_hb',
 'AIBot_DruidTrainee_002_hb',
 'AIBot_HunterTrainee_003_hb',
 'AIBot_MageTrainee_004_hb',
 'AIBot_PaladinTrainee_005_hb']

In [27]:
import sys
sys.path.append('../gym_hearthstone/envs/decks/classic')


In [67]:
from classic import classic_Zoo_Warlock

In [68]:
deck = classic_Zoo_Warlock.get_classic_zoo_warlock()
print(deck)

['EX1_308', 'EX1_319', 'EX1_302', 'CS2_189', 'EX1_310', 'CS2_188', 'EX1_029', 'EX1_004', 'EX1_162', 'NEW1_019', 'EX1_393', 'EX1_556', 'EX1_019', 'EX1_046', 'EX1_093', 'EX1_308', 'EX1_319', 'EX1_302', 'CS2_189', 'EX1_310', 'CS2_188', 'EX1_029', 'EX1_004', 'EX1_162', 'NEW1_019', 'EX1_393', 'EX1_556', 'EX1_019', 'EX1_046', 'EX1_093']


In [69]:
for card in deck:
    if card in implemented_cards:
        print("NICE!!")
    else:
        print("Press F to pay Respects")

NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!
NICE!!


In [35]:
c=cards.db[implemented_cards[0]]

In [36]:
c.id

'AIBot_DemonHunterTrainee_001_hb'

In [31]:
cards.db[implemented_cards[4726]]

<LOOT_008: 'Psychic Scream'>

In [39]:
for c in implemented_cards+unimplemented_cards:
    cards.db[c].atk
    cards.db[c].health

Todas las cartas son de tipo **hearthstone.cardxml.CardXML** y todas tienen los mismos atributos, sólo que unos tienen valor y otros no. Por ejemplo, DIVINE_SHIELD, las cartas de tipo SPELL no deberían tenerlo, pero siguen teniendo ese atributo puesto a False

In [41]:
for id in sorted(cards.db):
    card = cards.db[id]
    card.divine_shield 
    card.atk
    card.health

Importamos diferentes constantes de HEARTHSTONE

In [52]:
from hearthstone.enums import PlayState, Step, Mulligan, State, CardClass, Race, CardSet, CardType
list(PlayState)

[<PlayState.INVALID: 0>,
 <PlayState.PLAYING: 1>,
 <PlayState.WINNING: 2>,
 <PlayState.LOSING: 3>,
 <PlayState.WON: 4>,
 <PlayState.LOST: 5>,
 <PlayState.TIED: 6>,
 <PlayState.DISCONNECTED: 7>,
 <PlayState.CONCEDED: 8>]

In [21]:
list(Step)

[<Step.INVALID: 0>,
 <Step.BEGIN_FIRST: 1>,
 <Step.BEGIN_SHUFFLE: 2>,
 <Step.BEGIN_DRAW: 3>,
 <Step.BEGIN_MULLIGAN: 4>,
 <Step.MAIN_BEGIN: 5>,
 <Step.MAIN_READY: 6>,
 <Step.MAIN_RESOURCE: 7>,
 <Step.MAIN_DRAW: 8>,
 <Step.MAIN_START: 9>,
 <Step.MAIN_ACTION: 10>,
 <Step.MAIN_COMBAT: 11>,
 <Step.MAIN_END: 12>,
 <Step.MAIN_NEXT: 13>,
 <Step.FINAL_WRAPUP: 14>,
 <Step.FINAL_GAMEOVER: 15>,
 <Step.MAIN_CLEANUP: 16>,
 <Step.MAIN_START_TRIGGERS: 17>,
 <Step.MAIN_SET_ACTION_STEP_TYPE: 18>,
 <Step.MAIN_PRE_ACTION: 19>,
 <Step.MAIN_POST_ACTION: 20>]

In [22]:
list(Mulligan)

[<Mulligan.INVALID: 0>,
 <Mulligan.INPUT: 1>,
 <Mulligan.DEALING: 2>,
 <Mulligan.WAITING: 3>,
 <Mulligan.DONE: 4>]

In [23]:
list(State)

[<State.INVALID: 0>,
 <State.LOADING: 1>,
 <State.RUNNING: 2>,
 <State.COMPLETE: 3>]

In [24]:
list(CardClass)

[<CardClass.INVALID: 0>,
 <CardClass.DEATHKNIGHT: 1>,
 <CardClass.DRUID: 2>,
 <CardClass.HUNTER: 3>,
 <CardClass.MAGE: 4>,
 <CardClass.PALADIN: 5>,
 <CardClass.PRIEST: 6>,
 <CardClass.ROGUE: 7>,
 <CardClass.SHAMAN: 8>,
 <CardClass.WARLOCK: 9>,
 <CardClass.WARRIOR: 10>,
 <CardClass.DREAM: 11>,
 <CardClass.NEUTRAL: 12>,
 <CardClass.WHIZBANG: 13>,
 <CardClass.DEMONHUNTER: 14>]

In [25]:
for i in list(CardClass):
    print(i,"-",i.default_hero)

CardClass.INVALID - 
CardClass.DEATHKNIGHT - HERO_11
CardClass.DRUID - HERO_06
CardClass.HUNTER - HERO_05
CardClass.MAGE - HERO_08
CardClass.PALADIN - HERO_04
CardClass.PRIEST - HERO_09
CardClass.ROGUE - HERO_03
CardClass.SHAMAN - HERO_02
CardClass.WARLOCK - HERO_07
CardClass.WARRIOR - HERO_01
CardClass.DREAM - 
CardClass.NEUTRAL - 
CardClass.WHIZBANG - BOT_914h
CardClass.DEMONHUNTER - HERO_10


Parece que hay algunas CardClass que no tienen héroe asociado. Cuando hagamos un RANDOM para elegir los héroes de inicio, vamos a eliminar esas que no tengan un héroe asociado porque me da que puede dar un fallo más adelante. Voy a eliminarlas de las opciones

**comentario**
<br>
Probablemente haya que eliminar WhizBang también (No lo he testeado todavía pero creo que da error en Fireplace porque no está implementada)

In [32]:
heroes=list(CardClass)
heroes.remove(CardClass.INVALID)
heroes.remove(CardClass.DREAM)
heroes.remove(CardClass.NEUTRAL)
heroes.remove(CardClass.WHIZBANG)
heroes

[<CardClass.DEATHKNIGHT: 1>,
 <CardClass.DRUID: 2>,
 <CardClass.HUNTER: 3>,
 <CardClass.MAGE: 4>,
 <CardClass.PALADIN: 5>,
 <CardClass.PRIEST: 6>,
 <CardClass.ROGUE: 7>,
 <CardClass.SHAMAN: 8>,
 <CardClass.WARLOCK: 9>,
 <CardClass.WARRIOR: 10>,
 <CardClass.DEMONHUNTER: 14>]

In [27]:
list(Race)

[<Race.INVALID: 0>,
 <Race.BLOODELF: 1>,
 <Race.DRAENEI: 2>,
 <Race.DWARF: 3>,
 <Race.GNOME: 4>,
 <Race.GOBLIN: 5>,
 <Race.HUMAN: 6>,
 <Race.NIGHTELF: 7>,
 <Race.ORC: 8>,
 <Race.TAUREN: 9>,
 <Race.TROLL: 10>,
 <Race.UNDEAD: 11>,
 <Race.WORGEN: 12>,
 <Race.GOBLIN2: 13>,
 <Race.MURLOC: 14>,
 <Race.DEMON: 15>,
 <Race.SCOURGE: 16>,
 <Race.MECHANICAL: 17>,
 <Race.ELEMENTAL: 18>,
 <Race.OGRE: 19>,
 <Race.BEAST: 20>,
 <Race.TOTEM: 21>,
 <Race.NERUBIAN: 22>,
 <Race.PIRATE: 23>,
 <Race.DRAGON: 24>,
 <Race.BLANK: 25>,
 <Race.ALL: 26>,
 <Race.EGG: 38>,
 <Race.QUILBOAR: 43>,
 <Race.CENTAUR: 80>,
 <Race.FURBOLG: 81>,
 <Race.HIGHELF: 83>,
 <Race.TREANT: 84>,
 <Race.OWLKIN: 85>,
 <Race.HALFORC: 88>,
 <Race.LOCK: 89>,
 <Race.NAGA: 92>,
 <Race.OLDGOD: 93>,
 <Race.PANDAREN: 94>,
 <Race.GRONN: 95>,
 <Race.CELESTIAL: 96>,
 <Race.GNOLL: 97>,
 <Race.GOLEM: 98>,
 <Race.HARPY: 99>,
 <Race.VULPERA: 100>]

In [28]:
list(CardSet)

[<CardSet.INVALID: 0>,
 <CardSet.TEST_TEMPORARY: 1>,
 <CardSet.BASIC: 2>,
 <CardSet.EXPERT1: 3>,
 <CardSet.HOF: 4>,
 <CardSet.MISSIONS: 5>,
 <CardSet.DEMO: 6>,
 <CardSet.NONE: 7>,
 <CardSet.CHEAT: 8>,
 <CardSet.BLANK: 9>,
 <CardSet.DEBUG_SP: 10>,
 <CardSet.PROMO: 11>,
 <CardSet.NAXX: 12>,
 <CardSet.GVG: 13>,
 <CardSet.BRM: 14>,
 <CardSet.TGT: 15>,
 <CardSet.CREDITS: 16>,
 <CardSet.HERO_SKINS: 17>,
 <CardSet.TB: 18>,
 <CardSet.SLUSH: 19>,
 <CardSet.LOE: 20>,
 <CardSet.OG: 21>,
 <CardSet.OG_RESERVE: 22>,
 <CardSet.KARA: 23>,
 <CardSet.KARA_RESERVE: 24>,
 <CardSet.GANGS: 25>,
 <CardSet.GANGS_RESERVE: 26>,
 <CardSet.UNGORO: 27>,
 <CardSet.ICECROWN: 1001>,
 <CardSet.LOOTAPALOOZA: 1004>,
 <CardSet.GILNEAS: 1125>,
 <CardSet.BOOMSDAY: 1127>,
 <CardSet.TROLL: 1129>,
 <CardSet.DALARAN: 1130>,
 <CardSet.ULDUM: 1158>,
 <CardSet.DRAGONS: 1347>,
 <CardSet.YEAR_OF_THE_DRAGON: 1403>,
 <CardSet.BLACK_TEMPLE: 1414>,
 <CardSet.WILD_EVENT: 1439>,
 <CardSet.SCHOLOMANCE: 1443>,
 <CardSet.BATTLEGROUNDS: 1453

In [29]:
list(CardType)

[<CardType.INVALID: 0>,
 <CardType.GAME: 1>,
 <CardType.PLAYER: 2>,
 <CardType.HERO: 3>,
 <CardType.MINION: 4>,
 <CardType.SPELL: 5>,
 <CardType.ENCHANTMENT: 6>,
 <CardType.WEAPON: 7>,
 <CardType.ITEM: 8>,
 <CardType.TOKEN: 9>,
 <CardType.HERO_POWER: 10>,
 <CardType.BLANK: 11>,
 <CardType.GAME_MODE_BUTTON: 12>,
 <CardType.MOVE_MINION_HOVER_TARGET: 22>,
 <CardType.LETTUCE_ABILITY: 23>,
 <CardType.BATTLEGROUND_HERO_BUDDY: 24>,
 <CardType.LOCATION: 39>,
 <CardType.BATTLEGROUND_QUEST_REWARD: 40>]

Ahora vamos a replicar los pasos de la función **def setup_game(self)** para ver si funciona

In [33]:
hero1 = CardClass.MAGE
print(hero1)
print(hero1.default_hero)

CardClass.MAGE
HERO_08


Metemos en una función los comandos que van a elegir las cartas para un determinado héroe. En esas cartas vamos a meter las del héroe más las cartas neutras

In [34]:
[hero1, CardClass.NEUTRAL]

[<CardClass.MAGE: 4>, <CardClass.NEUTRAL: 12>]

In [35]:
def get_collection(heroe):
    collection = []
    for card in cards.db.keys():
        if str(card) not in implemented_cards:
                continue
        cls = cards.db[card]
        if not cls.collectible:
            continue
        if cls.type == CardType.HERO:
            # Heroes are collectible...
            continue
        if cls.card_class and cls.card_class in [heroe, CardClass.NEUTRAL]:
            # Play with more possibilities
            collection.append(cls)
    count=0
    for j in collection:
        if j.card_class == heroe:
            count=count+1
    print("Totales Neutras + hero: ",len(collection))
    print("Cartas de heroe: ",count)
    return collection

De entre esa collección vamos a elegir las 30 cartas del DECK de cada jugador. Ojo, parece que es posible que los dos players tengan cartas NEUTRAS repetidas ya que las cartas que se le han asignado a un jugador no se eliminan de las cartas que se le van a asignar al segundo jugador. Confirmar con JESUS que es así.

**comentario**
<br>
* El máximo número de cartas que se pueden llevar en cada mazo son 2 (No importa si son neutras o de clase)
* Lo que sí importa es la rareza de la carta (Común, Poco común o Rara, Épica y Legendaria)
* Solo se puede llevar un carta de las legendarias por considerarse más poderosas.

In [36]:
def get_deck(collection):
    deck=[]
    while len(deck) < Deck.MAX_CARDS:
        card = random.choice(collection)
        if deck.count(card.id) < card.max_count_in_deck:
            deck.append(card.id)
    return deck

In [37]:
collection1=get_collection(hero1)

Totales Neutras + hero:  867
Cartas de heroe:  128


In [38]:
deck1=get_deck(collection1)
print(deck1)

['EX1_162', 'GVG_094', 'LOOT_529', 'NEW1_041', 'KAR_029', 'EX1_062', 'TSC_065', 'EX1_620', 'EX1_508', 'CFM_806', 'LOOT_541', 'GIL_692', 'AT_070', 'OG_151', 'EX1_405', 'DAL_092', 'GIL_622', 'UNG_085', 'EX1_025', 'ICC_082', 'UNG_085', 'EX1_080', 'UNG_941', 'ICC_032', 'ICC_097', 'VAN_EX1_033', 'UNG_070', 'ICC_851', 'ICC_096', 'NEW1_017']


In [39]:
hero2=random.choice(heroes)
print(hero2)
collection2=get_collection(hero2)
deck2=get_deck(collection2)
print(deck2)

CardClass.DEATHKNIGHT
Totales Neutras + hero:  739
Cartas de heroe:  0
['DAL_090', 'CFM_852', 'OG_082', 'GIL_212', 'EX1_556', 'CFM_025', 'DAL_085', 'AT_111', 'CFM_806', 'EX1_006', 'GVG_114', 'CORE_EX1_017', 'OG_158', 'GVG_110', 'ICC_032', 'TSC_065', 'BOT_555', 'DAL_092', 'CS2_196', 'BOT_401', 'KAR_044', 'KAR_097', 'SCH_143', 'AT_115', 'ICC_913', 'BOT_907', 'CS2_182', 'BOT_562', 'ICC_018', 'LOOT_167']


In [40]:
from fireplace.player import Player
player1=Player("Player1", deck1, hero1.default_hero)
player2=Player("Player2", deck2, hero2.default_hero)

Vemos que inicialmente la mano y el deck de cada jugador están vacías:

In [41]:
player1.deck,player1.hand

(<Deck (0 cards)>, [])

In [42]:
player2.deck,player2.hand


(<Deck (0 cards)>, [])

# Inciar Juego

In [43]:
game=Game(players=(player1, player2))
game.start()

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Fireblast')>)> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Fireblast')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Jaina Proudmoore')>)> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Player1 summons [<Hero ('Jaina Proudmoore')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) shuffles their deck
[f

## Mulligan

In [44]:
player1.choice.cards

[<Minion ("Skycap'n Kragg")>,
 <Minion ('Secretkeeper')>,
 <Minion ("Tentacle of N'Zoth")>,
 <Minion ("Tol'vir Stoneshaper")>]

In [45]:
player2.choice.cards

[<Minion ('Dalaran Crusader')>,
 <Minion ("Sneed's Old Shredder")>,
 <Minion ('Fungalmancer')>]

In [46]:
player1.choice.choose()

[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) shuffles their deck


In [47]:
player2.choice.choose()

[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player2', hero=<Hero ('The Lich King')>) shuffles their deck
[fireplace.entity]: Player1 gets The Coin (GAME_005)
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Give(<Give.CARD>='GAME_005')> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Giving [<Spell ('The Coin')>] to Player1
[fireplace.card]: <Spell ('The Coin')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player2 begins turn 1
[fireplace.entity]: Player2 is now at 1 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ('The Lich King')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('The Lich King')>)]
[fireplace.entity]: Player2 draws <Minion ('Refreshment Vendor')>
[fireplace.card]

# Turno 1
a partir de aquí se pueden testear fácilmente las cartas

Usa este código para establecer el estado del juego

In [48]:
player1.max_mana = 10
player2.max_mana = 10

# limpiamos las manos
h = len(player1.hand)
for i in range(h):
    player1.hand[0].discard()
    
h = len(player2.hand)
for i in range (h):
    player2.hand[0].discard()


# limpiamos el field
h = len(player1.field)
for i in range(h):
    player1.field[0].discard()
    
h = len(player2.field)
for i in range (h):
    player2.field[0].discard()
    
# limpiamos secretos
h = len(player1.secrets)
for i in range(h):
    player1.secrets[0].discard()
    
h = len(player2.secrets)
for i in range (h):
    player2.secrets[0].discard()

[fireplace.entity]: Player1 is now at 10 mana crystals
[fireplace.entity]: Player2 is now at 10 mana crystals
[fireplace.entity]: Discarding <Minion ("Skycap'n Kragg")>
[fireplace.card]: <Minion ("Skycap'n Kragg")> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ('Secretkeeper')>
[fireplace.card]: <Minion ('Secretkeeper')> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ("Tentacle of N'Zoth")>
[fireplace.card]: <Minion ("Tentacle of N'Zoth")> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ("Tol'vir Stoneshaper")>
[fireplace.card]: <Minion ("Tol'vir Stoneshaper")> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Spell ('The Coin')>
[fireplace.card]: <Spell ('The Coin')> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ('Dalaran Crusader')>
[fireplace.card]: <Minion ('Dalaran Crusader')> moves from 

In [49]:
print(player1.field)
print(player2.field)
print(player1.secrets)
print(player2.secrets)
print(player1.hand)
print(player2.hand)

[]
[]
[]
[]
[]
[]


## Hero Power - map actions

In [80]:
game.end_turn()

[fireplace.entity]: Player2 ends turn 3
[fireplace.entity]: Player1 begins turn 4
[fireplace.entity]: Player1 is now at 10 mana crystals
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.entity]: Player1 draws <Minion ('Lifedrinker')>
[fireplace.card]: <Minion ('Lifedrinker')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[None]

In [78]:
player2.give("EX1_029").play()

[fireplace.actions]: Player(name='Player2', hero=<Hero ('The Lich King')>) triggering <TargetedAction: Give(<Give.CARD>='EX1_029')> targeting [Player(name='Player2', hero=<Hero ('The Lich King')>)]
[fireplace.actions]: Giving [<Minion ('Leper Gnome')>] to Player2
[fireplace.card]: <Minion ('Leper Gnome')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player2 plays <Minion ('Leper Gnome')> (target=None, index=None)
[fireplace.entity]: Player2 pays 1 mana
[fireplace.card]: <Minion ('Leper Gnome')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Leper Gnome')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Leper Gnome')>]
[fireplace.actions]: Activating <Minion ('Leper Gnome')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processi

<Minion ('Leper Gnome')>

In [69]:
player1.give("EX1_029").play()

[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Give(<Give.CARD>='EX1_029')> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Giving [<Minion ('Leper Gnome')>] to Player1
[fireplace.card]: <Minion ('Leper Gnome')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


InvalidAction: <Minion ('Leper Gnome')> isn't playable.

In [79]:
player1.hero.power.targets

[<Hero ('Jaina Proudmoore')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Hero ('The Lich King')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>,
 <Minion ('Leper Gnome')>]

In [83]:
player1.hero.power.targets[1].type

4

## Análisis del bucle Hero Power

In [96]:
# LOE_076 Sr Finley (Hero Power Change)
finley = player1.give("LOE_076").play()

[fireplace.actions]: Player(name='Player1', hero=<Hero ('Uther Lightbringer')>) triggering <TargetedAction: Give(<Give.CARD>='LOE_076')> targeting [Player(name='Player1', hero=<Hero ('Uther Lightbringer')>)]
[fireplace.actions]: Giving [<Minion ('Sir Finley Mrrgglton')>] to Player1
[fireplace.card]: <Minion ('Sir Finley Mrrgglton')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Minion ('Sir Finley Mrrgglton')> (target=None, index=None)
[fireplace.entity]: Player1 pays 1 mana
[fireplace.card]: <Minion ('Sir Finley Mrrgglton')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Sir Finley Mrrgglton')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Sir Finley Mrrgglton')>]
[fireplace.actions]: Activating <Minion ('Sir Finley Mrrgglton')> action targeting None
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Uthe

In [97]:
player1.choice.cards

[<HeroPower ('Demon Claws')>,
 <HeroPower ('Dagger Mastery')>,
 <HeroPower ('Totemic Call')>]

In [98]:
player1.choice.choose(player1.choice.cards[2])

[fireplace.entity]: Discarding <HeroPower ('Demon Claws')>
[fireplace.card]: <HeroPower ('Demon Claws')> moves from <Zone.SETASIDE: 6> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <HeroPower ('Dagger Mastery')>
[fireplace.card]: <HeroPower ('Dagger Mastery')> moves from <Zone.SETASIDE: 6> to <Zone.GRAVEYARD: 4>
[fireplace.actions]: <HeroPower ('Totemic Call')> triggering <TargetedAction: Destroy()> targeting [<HeroPower ('Totemic Call')>]
[fireplace.actions]: <HeroPower ('Totemic Call')> destroys <HeroPower ('Totemic Call')>
[fireplace.card]: <HeroPower ('Totemic Call')> moves from <Zone.PLAY: 1> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.card]: <HeroPower ('Totemic Call')> moves from <Zone.SETASIDE: 6> to <Zone.PLAY: 1>


## Error con Living Monument (Enchantment)

In [29]:
living_monument = player1.give("ULD_193").play()

[fireplace.actions]: Player(name='Player1', hero=<Hero ('Uther Lightbringer')>) triggering <TargetedAction: Give(<Give.CARD>='ULD_193')> targeting [Player(name='Player1', hero=<Hero ('Uther Lightbringer')>)]
[fireplace.actions]: Giving [<Minion ('Living Monument')>] to Player1
[fireplace.card]: <Minion ('Living Monument')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Minion ('Living Monument')> (target=None, index=None)
[fireplace.entity]: Player1 pays 10 mana
[fireplace.card]: <Minion ('Living Monument')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Living Monument')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Living Monument')>]
[fireplace.actions]: Activating <Minion ('Living Monument')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Emp

In [45]:
print(living_monument.type)
print(living_monument.atk)
print(living_monument.health)

4
6
2


In [31]:
print(living_monument in implemented_cards)

True


In [53]:
game.end_turn()

[fireplace.entity]: Player1 ends turn 3
[fireplace.entity]: Player2 begins turn 4
[fireplace.entity]: Player2 is now at 10 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Malfurion Stormrage')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('Malfurion Stormrage')>)]
[fireplace.entity]: Player2 draws <Minion ('Ravenholdt Assassin')>
[fireplace.card]: <Minion ('Ravenholdt Assassin')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[None]

In [54]:
voljin = player2.give("GVG_014")
voljin.targets

[fireplace.actions]: Player(name='Player2', hero=<Hero ('Malfurion Stormrage')>) triggering <TargetedAction: Give(<Give.CARD>='GVG_014')> targeting [Player(name='Player2', hero=<Hero ('Malfurion Stormrage')>)]
[fireplace.actions]: Giving [<Minion ("Vol'jin")>] to Player2
[fireplace.card]: <Minion ("Vol'jin")> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[<Minion ('Living Monument')>, <Minion ("Vol'jin")>]

In [55]:
voljin.play(voljin.targets[1])

[fireplace.actions]: Player2 plays <Minion ("Vol'jin")> (target=<Minion ("Vol'jin")>, index=None)
[fireplace.entity]: Player2 pays 5 mana
[fireplace.card]: <Minion ("Vol'jin")> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ("Vol'jin")> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=<Minion ("Vol'jin")>)> targeting [<Minion ("Vol'jin")>]
[fireplace.actions]: Activating <Minion ("Vol'jin")> action targeting <Minion ("Vol'jin")>
[fireplace.actions]: <Minion ("Vol'jin")> triggering <TargetedAction: SwapHealth(<SwapHealth.OTHER>=<fireplace.dsl.selector.FuncSelector object at 0x0000029875DFDEB0>, <SwapHealth.BUFF>='GVG_014a')> targeting [<Minion ("Vol'jin")>]
[fireplace.entity]: Applying <Enchantment ('Shadowed')> to <Minion ("Vol'jin")>
[fireplace.entity]: <Enchantment ('Shadowed')> removes all damage from <Minion ("Vol'jin")>
[fireplace.card]: <Enchantment ('Shadowed')> moves from <Zone.SETASIDE: 6> to <Zone.PLAY: 1>
[fireplace.entity]: Applying <Enchan

<Minion ("Vol'jin")>

In [42]:
voljin.is_playable()

False

In [56]:
p1 = player1
p2 = player2

print(len(p1.field))
print(len(p2.field))
p2.field

1
2


[<Minion ("Vol'jin")>, <Minion ("Vol'jin")>]

In [58]:
p2.field[0].health if 0 < len(p2.field) and p2.field[0].type != 7 and p2.field[0].type !=5 else 0

2

In [59]:
p2.field[1].health if 1 < len(p2.field) and p2.field[1].type != 7 and p2.field[1].type !=5 else 0

10

In [57]:
p1.field[0].health if 0 < len(p1.field) and p1.field[0].type != 7 and p1.field[0].type !=5 else 0

2

In [130]:
for i in range(0):
    print("xd")

## Análisis de la carta NEW1_038 - Gruul

In [125]:
gruul = player1.give("NEW1_038").play()



[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Give(<Give.CARD>='NEW1_038')> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Giving [<Minion ('Gruul')>] to Player1
[fireplace.card]: <Minion ('Gruul')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Minion ('Gruul')> (target=None, index=None)
[fireplace.entity]: Player1 pays 8 mana
[fireplace.card]: <Minion ('Gruul')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Gruul')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Gruul')>]
[fireplace.actions]: Activating <Minion ('Gruul')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


In [126]:
gruul.type

4

In [127]:
game.end_turn()

[fireplace.actions]: <Minion ('Gruul')> triggers off <Action: EndTurn(<EndTurn.PLAYER>=Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>))> from Game(players=(Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>), Player(name='Player2', hero=<Hero ("Gul'dan")>)))
[fireplace.actions]: <Minion ('Gruul')> triggering <TargetedAction: Buff(<Buff.BUFF>='NEW1_038o')> targeting [<Minion ('Gruul')>]
[fireplace.entity]: Applying <Enchantment ('Growth')> to <Minion ('Gruul')>
[fireplace.card]: <Enchantment ('Growth')> moves from <Zone.SETASIDE: 6> to <Zone.PLAY: 1>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player1 ends turn 4
[fireplace.entity]: Player2 begins turn 5
[fireplace.entity]: Player2 is now at 10 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ("Gul'dan")>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ("Gul'dan")>)]
[fireplace.entity]: Player2 draws <Minion ('Antique 

[None]

In [128]:
gruul.type

4

## Análisis de Secretos

In [99]:
# player 1 juega el counter spell (secreto)
player1.give("EX1_287").play()
game.end_turn()



[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Give(<Give.CARD>='EX1_287')> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Giving [<Secret ('Counterspell')>] to Player1
[fireplace.card]: <Secret ('Counterspell')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Secret ('Counterspell')> (target=None, index=None)
[fireplace.entity]: Player1 pays 3 mana
[fireplace.card]: <Secret ('Counterspell')> moves from <Zone.HAND: 3> to <Zone.SECRET: 7>
[fireplace.actions]: <Secret ('Counterspell')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Secret ('Counterspell')>]
[fireplace.actions]: Activating <Secret ('Counterspell')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing

[None]

In [103]:
# player 2 juega bola de fuego (hechizo)
fireball = player2.give("CS2_029")
print(fireball.targets)

[fireplace.actions]: Player(name='Player2', hero=<Hero ("Gul'dan")>) triggering <TargetedAction: Give(<Give.CARD>='CS2_029')> targeting [Player(name='Player2', hero=<Hero ("Gul'dan")>)]
[fireplace.actions]: Giving [<Spell ('Fireball')>] to Player2
[fireplace.card]: <Spell ('Fireball')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[<Hero ('Garrosh Hellscream')>, <Hero ("Gul'dan")>, <Minion ('Silverback Patriarch')>, <Minion ('Silverback Patriarch')>, <Minion ('Silverback Patriarch')>, <Minion ('Silverback Patriarch')>]


In [105]:
fireball.play(fireball.targets[0])

[fireplace.actions]: Player2 plays <Spell ('Fireball')> (target=<Hero ('Garrosh Hellscream')>, index=None)
[fireplace.entity]: Player2 pays 4 mana
[fireplace.card]: <Spell ('Fireball')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Secret ('Counterspell')> triggers off <Action: Play(<Play.PLAYER>=<Spell ('Fireball')>, <Play.CARD>=<Hero ('Garrosh Hellscream')>, <Play.TARGET>=None, <Play.INDEX>=None)> from Player(name='Player2', hero=<Hero ("Gul'dan")>)
[fireplace.actions]: <Secret ('Counterspell')> triggering <TargetedAction: Reveal()> targeting [<Secret ('Counterspell')>]
[fireplace.actions]: Revealing secret <Secret ('Counterspell')>
[fireplace.card]: <Secret ('Counterspell')> moves from <Zone.SECRET: 7> to <Zone.GRAVEYARD: 4>
[fireplace.actions]: <Secret ('Counterspell')> triggering <TargetedAction: Counter()> targeting [<Spell ('Fireball')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras a

<Spell ('Fireball')>

In [108]:
print(player1.field)
print(player2.field)
print(player1.secrets)
print(player2.secrets)

[]
[]
[]
[]


In [ ]:
# pasa turno
game.end_turn()

# player 2 juega bola de fuego (hechizo)
player2.give().play()
game.end_turn()

In [ ]:
print(player1.field)
print(player2.field)
print(player2.secrets)

## Resolviendo error Enchantment
<hr>

In [118]:
# player 1 juega el gnomo
player1.give("EX1_029").play()
game.end_turn()

[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Give(<Give.CARD>='EX1_029')> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Giving [<Minion ('Leper Gnome')>] to Player1
[fireplace.card]: <Minion ('Leper Gnome')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player1 plays <Minion ('Leper Gnome')> (target=None, index=None)
[fireplace.entity]: Player1 pays 1 mana
[fireplace.card]: <Minion ('Leper Gnome')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Leper Gnome')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Leper Gnome')>]
[fireplace.actions]: Activating <Minion ('Leper Gnome')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras an

[None]

In [119]:
# player 2 juega un bicho y el secreto Bad Tactics
player2.give("CS2_127").play()
player2.give("BT_203").play()
game.end_turn()

[fireplace.actions]: Player(name='Player2', hero=<Hero ("Gul'dan")>) triggering <TargetedAction: Give(<Give.CARD>='CS2_127')> targeting [Player(name='Player2', hero=<Hero ("Gul'dan")>)]
[fireplace.actions]: Giving [<Minion ('Silverback Patriarch')>] to Player2
[fireplace.card]: <Minion ('Silverback Patriarch')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player2 plays <Minion ('Silverback Patriarch')> (target=None, index=None)
[fireplace.entity]: Player2 pays 3 mana
[fireplace.card]: <Minion ('Silverback Patriarch')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Silverback Patriarch')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Silverback Patriarch')>]
[fireplace.actions]: Activating <Minion ('Silverback Patriarch')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.

[None]

In [120]:
print(player1.field)
print(player2.field)
print(player2.secrets)

[<Minion ('Leper Gnome')>]
[<Minion ('Silverback Patriarch')>]
[<Secret ('Pack Tactics')>]


In [121]:
# Targets of the gnome
gnome = player1.field[0]
targets = gnome.targets
print(targets)

[<Minion ('Silverback Patriarch')>]


In [122]:
# Atacamos al esbirro para activar el secreto Bad Tactics
gnome.attack(targets[0])

[fireplace.actions]: <Minion ('Leper Gnome')> attacks <Minion ('Silverback Patriarch')>
[fireplace.actions]: <Secret ('Pack Tactics')> triggers off <Action: Attack(<Attack.ATTACKER>=<Minion ('Leper Gnome')>, <Attack.DEFENDER>=<Minion ('Silverback Patriarch')>)> from <Minion ('Leper Gnome')>
[fireplace.actions]: <Secret ('Pack Tactics')> triggering <TargetedAction: Summon(<Summon.CARD>=ExactCopy(<Attack.DEFENDER>))> targeting [Player(name='Player2', hero=<Hero ("Gul'dan")>)]
[fireplace.copy]: Creating a copy of <Minion ('Silverback Patriarch')>
[fireplace.actions]: Player2 summons [<Minion ('Silverback Patriarch')>]
[fireplace.card]: <Minion ('Silverback Patriarch')> moves from <Zone.SETASIDE: 6> to <Zone.PLAY: 1>
[fireplace.actions]: <TargetedAction: Summon(<Summon.CARD>=ExactCopy(<Attack.DEFENDER>))> queues up callback <TargetedAction: Buff(<Buff.BUFF>='BT_203e')>
[fireplace.actions]: <Secret ('Pack Tactics')> triggering <TargetedAction: Buff(<Buff.BUFF>='BT_203e')> targeting [<Minion

In [123]:
print(player1.field)
print(player2.field)
print(player2.secrets)

[]
[<Minion ('Silverback Patriarch')>, <Minion ('Silverback Patriarch')>]
[<Secret ('Pack Tactics')>]


In [124]:
print(player2.field[0].atk)
print(player2.field[0].health)
print(player2.field[0].type)
print("")
print(player2.field[1].atk)
print(player2.field[1].health)
print(player2.field[1].type)


1
2
4

3
3
4


## Conclusiones

    - El bicho creado se identifica como esbirro (Puedes pedirle ataque y defensa).
    - Cuando el bicho sigue vivo también se identifica como esbirro.
    - El SECRETO sigue activo después de usarlo (debería desaparecer)!!!
    
Esto es una contradicción de lo que he estado viendo...

### Resolviendo error de Kazakus (Bucle infinito)
<hr>

In [73]:
player2.give("CFM_621")

[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Give(<Give.CARD>='CFM_621')> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Giving [<Minion ('Kazakus')>] to Player2
[fireplace.card]: <Minion ('Kazakus')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


<Minion ('Kazakus')>

In [79]:
game.end_turn()

[fireplace.entity]: Player1 ends turn 6
[fireplace.entity]: Player2 begins turn 7
[fireplace.entity]: Player2 is now at 4 mana crystals
[fireplace.actions]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Draw()> targeting [Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.entity]: Player2 draws <Minion ('Corpsetaker')>
[fireplace.card]: <Minion ('Corpsetaker')> moves from <Zone.DECK: 2> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[None]

In [80]:
player2.hand[4].play()

[fireplace.actions]: Player2 plays <Minion ('Kazakus')> (target=None, index=None)
[fireplace.entity]: Player2 pays 4 mana
[fireplace.card]: <Minion ('Kazakus')> moves from <Zone.HAND: 3> to <Zone.PLAY: 1>
[fireplace.actions]: <Minion ('Kazakus')> triggering <TargetedAction: Battlecry(<Battlecry.TARGET>=None)> targeting [<Minion ('Kazakus')>]
[fireplace.actions]: Activating <Minion ('Kazakus')> action targeting None
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


<Minion ('Kazakus')>

In [85]:
player2.choice.cards

[<Spell ('Felbloom')>, <Spell ('Stonescale Oil')>, <Spell ('Icecap')>]

In [86]:
print(player2.choice.cards[0])
player2.choice.choose(player2.choice.cards[0])

Felbloom


KeyError: 'CFM_621t1'

### Análisis de los tipos de Cartas
<hr>

In [37]:
player1.deck,player1.hand

(<Deck (26 cards)>,
 [<Minion ('Worgen Abomination')>,
  <Spell ('Naturalize')>,
  <Minion ('Dendrologist')>,
  <Minion ("C'Thun's Chosen")>,
  <Spell ('The Coin')>])

In [40]:
player2.deck,player2.hand

(<Deck (26 cards)>,
 [<Minion ('Rotface')>,
  <Minion ('Faerie Dragon')>,
  <Minion ('Recruiter')>,
  <Minion ('Axe Flinger')>,
  <Minion ('Kazakus')>])

In [59]:
print(player2.hero)
player2.give("EX1_567")
player2.give("EX1_536")

[fireplace.actions]: Player(name='Player2', hero=<Hero ('Thrall')>) triggering <TargetedAction: Give(<Give.CARD>='EX1_536')> targeting [Player(name='Player2', hero=<Hero ('Thrall')>)]
[fireplace.actions]: Giving [<Weapon ('Eaglehorn Bow')>] to Player2
[fireplace.card]: <Weapon ('Eaglehorn Bow')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


Thrall


<Weapon ('Eaglehorn Bow')>

In [77]:
weapon0 = player2.hand[3]
# Check Windfury
print(weapon0.windfury)
# Check Aura
print(weapon0.aura)
# Check Deathrattle
print(weapon0.has_deathrattle)


## Las armas no tienen los siguientes atributos
# divine_shield
# charge
# taunt
# stealthed
# poisonous
# cantbetarget
# frozen
# silenced

1
False
False


### Corrección del error AttributeError: 'Spell' object has no attribute 'divine_shield'

La función **def _get_state(self)** da error cuando salen algunas cartas, por ejemplo la carta "Lock and Load" que es de tipo SPELL. Cuando se va a calcular el estado y se mira el valor del atributo divine_shield, obtenemos el error:

    AttributeError: 'Spell' object has no attribute 'divine_shield'
    
Cuando en realidad en las cartas de la librería HEARTHSTONE todas tienen ese atributo. Me da que Fireplace debe crear clases diferentes para cartas diferentes...

En esta ejecución veo que al jugador 2 le ha tocado un:

    'fireplace.card.Spell' Ice Fishing
    
Si esa carta le hubiera tocado al jugador 1, al ejecutar la función _get_state(self) tal y como está definida, debería dar otro error de ese tipo:

    "myhand2effects_divineshield": 1 if 0 < len(p1.hand) and p1.hand[0].divine_shield else 0,

[Recordar que la función que devuelve el status no puede devolver la mano del contrincante que teóricamente es desconocida]

Vamos a aislar ese SPELL para ver qué atributos tiene de entre estos:

"myhand0effects_windfury":        SI  
"myhand0effects_divineshield":    NO  
"myhand0effects_charge":          NO  
"myhand0effects_taunt":           NO  
"myhand0effects_stealth":         NO  
"myhand0effects_poisonous":       NO  
"myhand0effects_cantbetargeted":  NO  
"myhand0effects_aura":            SI  
"myhand0effects_deathrattle":     NO  
"myhand0effects_frozen":          NO  
"myhand0effects_silenced":        NO  

Lo que me lleva a dos preguntas:

* Esto es así sólo en <Spell ('Ice Fishing')> o puede haber otros SPELLS que sí que los tengan??
* Puede que tengamos que añadir otros efectos de los SPELLs para que el get_status esté más completo? ¿Sólo se han incluido estos por simplificar o porque no hay mas (windfury,divineshield, charge, taunt, stealth, poisonous, cantbetargeted,aura, deathratttle, frozen, silenced)?

PREGUNTAR A JESUS

In [157]:
player2.hand[2]

<Spell ('Ice Fishing')>

In [189]:
c=player2.hand[2]
c.divine_shield

AttributeError: 'Spell' object has no attribute 'divine_shield'

Si la respuesta a las otras dos preguntas es que ningún SPELL tiene esos atributos, y tampoco hay que añadir atributos nuevos, podríamos resolver el problema:
    
    AttributeError: 'Spell' object has no attribute 'divine_shield'
        
Modificando la línea:
    
    "myhand2effects_divineshield": 1 if 0 < len(p1.hand) and p1.hand[0].divine_shield else 0,
    
Por:

In [190]:
def get_divine_shield(card):
    try:
        return card.divine_shield
    except AttributeError:
        return 0      

In [202]:
1 if 2 < len(p2.hand) and get_divine_shield(p2.hand[2]) else 0

0

Notar que sólo devuelve 1 si el valor del atributo divine_shield es diferente de 0. Si esa carta no tiene divine_shield, como es el caso de los SPELL o lo tiene pero vale 0, sigue devolviendo 0. Entiendo que esto es correcto.

CONSULTAR CON JESUS

El problema es que cada vez que haya que invocar a get_status va a tener que llamar un montón de veces a estas funciones. Creo que va a ralentizar mucho el proceso. Lo suyo sería modificar la clase SPELL de fireplace para añadirle los atributos que ahora mismo no tiene. O bien, meter una condición que detecte si es un objeto de la clase SPELL... Algo como esto:

In [203]:
1 if 2 < len(p2.hand) and p2.hand[2].type!=5 and p2.hand[2].divine_shield  else 0

0

Esta idea me gusta más. Habrá que ver más adelante, si hay otras cartas que no tengan atributos e ir metiendo condiciones cada vez más complicadas. En este caso, la condición:

    p2.hand[2].type!=5
    
Chequea si es una carta de tipo SPELL. Pero puede que haga falta chequear si hay cartas de otros tipos.

**comentario**
<br>
Contesto a todo aquí al final que va ser más ordenado.
<br>
En principio las cartas de clase SPELL no tienen los atributos de los esbirros. 
No tienen *Attack*, ni *Divine_shield*, ni *Windfury* ... etc
<br>
Los SPELLS, sin embargo, si pueden dar estos attributos a un esbirro. (Puede ser su 'efecto')
<br>
Por ejemplo, un spell puede darle *Windfury* o *Divine_shield* a uno varios esbirros. 
<br>
<br>
No estoy seguro de cómo lo implementa Fireplace pero por lo que he visto yo creo que los Spells
no tienen ninguno de estos atributos. Lo que no entiendo es porque el error salta solo con *Divine_shield*
y no con otros atributos como *Windfury*.
<br>

Y para terminar los spells no son las únicas cartas que no tienen estos elementos:

    type = 7 , que son las cartas 'Armas' pueden tener Windfury pero no Divine_shield

Quizá teniendo eso en cuenta el fireplace gestiona *Windfury* de otra manera.
<br>
Lo que está claro es que la función que has creado debería funcionar para todas las cartas sin Divine_shield.
<br>
<br>
Acabo de comprobar que has optado por comprobar que no es 'type = 5', así que para ser consistente añadiré también type = 7. Como las armas (type 7) sí tienen algunos atributos y no estoy seguro de como lo implementa Fireplace por ahora solo se lo añado a divine_shield.

Para comprobar que atributos funcionan con armas y cuales no voy a crear una carta de arma y probar el código
    
    Arma normal: Eaglehorn bow -> EX1_536 (Card code)
    Arma con Windfury: Doomhammer -> EX1_567 (Card code)

In [154]:
p1=player1
p2=player2

In [156]:
for i in range(10):
    try:
        print(">>>>",type(p2.hand[i]),p2.hand[i])
    except:
        print(">>>> no ",i)

>>>> <class 'fireplace.card.Minion'> River Crocolisk
>>>> <class 'fireplace.card.Minion'> Drakkari Enchanter
>>>> <class 'fireplace.card.Spell'> Ice Fishing
>>>> no  3
>>>> no  4
>>>> no  5
>>>> no  6
>>>> no  7
>>>> no  8
>>>> no  9


## Expresiones regulares

In [33]:
for id in sorted(cards.db):
    card = cards.db[id]
    ret = card.description
    ret_org=ret
    ret=ret.replace("\n"," ") 
    ret = re.sub("<i>.+</i>", "", ret,flags=re.IGNORECASE)
    ret = re.sub(r"<\/?b[^>]*>", "", ret)
    ret = re.sub("(" + "|".join(SOLVED_KEYWORDS) + ")", "", ret)
    ret2 = re.sub("<[^>]*>", "", ret)

## Class Autonum

Para qué vale??

In [ ]:
from enum import Enum

class AutoNumber(Enum):
	def __new__(cls):
		value = len(cls.__members__)  # note no + 1
		obj = object.__new__(cls)
		obj._value_ = value
		return obj

class Move(AutoNumber):
	end_turn = ()
	hero_power = ()
	minion_attack = ()
	hero_attack = ()
	play_card = ()
	mulligan = ()
	choice = ()

## Modify hearthstone environment from nested to unnested

In [241]:
from gymnasium import spaces
modify={"myhand0effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand1effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand2effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand3effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand4effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand5effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand6effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand7effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand8effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myhand9effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield0effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield1effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield2effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield3effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield4effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield5effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "myfield6effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield0effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield1effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield2effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield3effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield4effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield5effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            ),
            "oppfield6effects": spaces.Dict({
                "windfury": spaces.Discrete(2),
                "divineshield": spaces.Discrete(2),
                "charge": spaces.Discrete(2),
                "taunt": spaces.Discrete(2),
                "stealth": spaces.Discrete(2),
                "poisonous": spaces.Discrete(2),
                "cantbetargeted": spaces.Discrete(2),
                "aura": spaces.Discrete(2),
                "deathrattle": spaces.Discrete(2),
                "frozen": spaces.Discrete(2),
                "silenced": spaces.Discrete(2)}
            )}

In [242]:
modify["myhand0effects"].keys()

KeysView(Dict('aura': Discrete(2), 'cantbetargeted': Discrete(2), 'charge': Discrete(2), 'deathrattle': Discrete(2), 'divineshield': Discrete(2), 'frozen': Discrete(2), 'poisonous': Discrete(2), 'silenced': Discrete(2), 'stealth': Discrete(2), 'taunt': Discrete(2), 'windfury': Discrete(2)))

In [243]:
for k,v in modify.items():
    for kk,vv in v.items():
        print('"'+k+"_"+kk+'": spaces.Discrete(2),')

"myhand0effects_aura": spaces.Discrete(2),
"myhand0effects_cantbetargeted": spaces.Discrete(2),
"myhand0effects_charge": spaces.Discrete(2),
"myhand0effects_deathrattle": spaces.Discrete(2),
"myhand0effects_divineshield": spaces.Discrete(2),
"myhand0effects_frozen": spaces.Discrete(2),
"myhand0effects_poisonous": spaces.Discrete(2),
"myhand0effects_silenced": spaces.Discrete(2),
"myhand0effects_stealth": spaces.Discrete(2),
"myhand0effects_taunt": spaces.Discrete(2),
"myhand0effects_windfury": spaces.Discrete(2),
"myhand1effects_aura": spaces.Discrete(2),
"myhand1effects_cantbetargeted": spaces.Discrete(2),
"myhand1effects_charge": spaces.Discrete(2),
"myhand1effects_deathrattle": spaces.Discrete(2),
"myhand1effects_divineshield": spaces.Discrete(2),
"myhand1effects_frozen": spaces.Discrete(2),
"myhand1effects_poisonous": spaces.Discrete(2),
"myhand1effects_silenced": spaces.Discrete(2),
"myhand1effects_stealth": spaces.Discrete(2),
"myhand1effects_taunt": spaces.Discrete(2),
"myhand1e

### Modify hearthstone environment from nested to unnested

Y también modificado para añadir la condición de no buscar ciertos atributos cuando la carta es de tipo SPELL

In [256]:
code="""""myhand0effects_windfury": 1 if 0 < len(p1.hand) and p1.hand[0].windfury else 0,
"myhand0effects_divineshield": 1 if 0 < len(p1.hand) and p1.hand[0].divine_shield else 0,
"myhand0effects_charge": 1 if 0 < len(p1.hand) and p1.hand[0].charge else 0,
"myhand0effects_taunt": 1 if 0 < len(p1.hand) and p1.hand[0].taunt else 0,
"myhand0effects_stealth": 1 if 0 < len(p1.hand) and p1.hand[0].stealthed else 0,
"myhand0effects_poisonous": 1 if 0 < len(p1.hand) and p1.hand[0].poisonous else 0,
"myhand0effects_cantbetargeted": 1 if 0 < len(p1.hand) and (p1.hand[0].cant_be_targeted_by_abilities or p1.hand[0].cant_be_targeted_by_hero_powers) else 0,
"myhand0effects_aura": 1 if 0 < len(p1.hand) and p1.hand[0].aura else 0,
"myhand0effects_deathrattle": 1 if 0 < len(p1.hand) and p1.hand[0].has_deathrattle else 0,
"myhand0effects_frozen": 1 if 0 < len(p1.hand) and p1.hand[0].frozen else 0,
"myhand0effects_silenced": 1 if 0 < len(p1.hand) and p1.hand[0].silenced else 0,
"myhand1effects_windfury": 1 if 1 < len(p1.hand) and p1.hand[1].windfury else 0,
"myhand1effects_divineshield": 1 if 1 < len(p1.hand) and p1.hand[1].divine_shield else 0,
"myhand1effects_charge": 1 if 1 < len(p1.hand) and p1.hand[1].charge else 0,
"myhand1effects_taunt": 1 if 1 < len(p1.hand) and p1.hand[1].taunt else 0,
"myhand1effects_stealth": 1 if 1 < len(p1.hand) and p1.hand[1].stealthed else 0,
"myhand1effects_poisonous": 1 if 1 < len(p1.hand) and p1.hand[1].poisonous else 0,
"myhand1effects_cantbetargeted": 1 if 1 < len(p1.hand) and (p1.hand[1].cant_be_targeted_by_abilities or p1.hand[1].cant_be_targeted_by_hero_powers) else 0,
"myhand1effects_aura": 1 if 1 < len(p1.hand) and p1.hand[1].aura else 0,
"myhand1effects_deathrattle": 1 if 1 < len(p1.hand) and p1.hand[1].has_deathrattle else 0,
"myhand1effects_frozen": 1 if 1 < len(p1.hand) and p1.hand[1].frozen else 0,
"myhand1effects_silenced": 1 if 1 < len(p1.hand) and p1.hand[1].silenced else 0,
"myhand2effects_windfury": 1 if 2 < len(p1.hand) and p1.hand[2].windfury else 0,
"myhand2effects_divineshield": 1 if 2 < len(p1.hand) and p1.hand[2].divine_shield else 0,
"myhand2effects_charge": 1 if 2 < len(p1.hand) and p1.hand[2].charge else 0,
"myhand2effects_taunt": 1 if 2 < len(p1.hand) and p1.hand[2].taunt else 0,
"myhand2effects_stealth": 1 if 2 < len(p1.hand) and p1.hand[2].stealthed else 0,
"myhand2effects_poisonous": 1 if 2 < len(p1.hand) and p1.hand[2].poisonous else 0,
"myhand2effects_cantbetargeted": 1 if 2 < len(p1.hand) and (p1.hand[2].cant_be_targeted_by_abilities or p1.hand[2].cant_be_targeted_by_hero_powers) else 0,
"myhand2effects_aura": 1 if 2 < len(p1.hand) and p1.hand[2].aura else 0,
"myhand2effects_deathrattle": 1 if 2 < len(p1.hand) and p1.hand[2].has_deathrattle else 0,
"myhand2effects_frozen": 1 if 2 < len(p1.hand) and p1.hand[2].frozen else 0,
"myhand2effects_silenced": 1 if 2 < len(p1.hand) and p1.hand[2].silenced else 0,
"myhand3effects_windfury": 1 if 3 < len(p1.hand) and p1.hand[3].windfury else 0,
"myhand3effects_divineshield": 1 if 3 < len(p1.hand) and p1.hand[3].divine_shield else 0,
"myhand3effects_charge": 1 if 3 < len(p1.hand) and p1.hand[3].charge else 0,
"myhand3effects_taunt": 1 if 3 < len(p1.hand) and p1.hand[3].taunt else 0,
"myhand3effects_stealth": 1 if 3 < len(p1.hand) and p1.hand[3].stealthed else 0,
"myhand3effects_poisonous": 1 if 3 < len(p1.hand) and p1.hand[3].poisonous else 0,
"myhand3effects_cantbetargeted": 1 if 3 < len(p1.hand) and (p1.hand[3].cant_be_targeted_by_abilities or p1.hand[3].cant_be_targeted_by_hero_powers) else 0,
"myhand3effects_aura": 1 if 3 < len(p1.hand) and p1.hand[3].aura else 0,
"myhand3effects_deathrattle": 1 if 3 < len(p1.hand) and p1.hand[3].has_deathrattle else 0,
"myhand3effects_frozen": 1 if 3 < len(p1.hand) and p1.hand[3].frozen else 0,
"myhand3effects_silenced": 1 if 3 < len(p1.hand) and p1.hand[3].silenced else 0,
"myhand4effects_windfury": 1 if 4 < len(p1.hand) and p1.hand[4].windfury else 0,
"myhand4effects_divineshield": 1 if 4 < len(p1.hand) and p1.hand[4].divine_shield else 0,
"myhand4effects_charge": 1 if 4 < len(p1.hand) and p1.hand[4].charge else 0,
"myhand4effects_taunt": 1 if 4 < len(p1.hand) and p1.hand[4].taunt else 0,
"myhand4effects_stealth": 1 if 4 < len(p1.hand) and p1.hand[4].stealthed else 0,
"myhand4effects_poisonous": 1 if 4 < len(p1.hand) and p1.hand[4].poisonous else 0,
"myhand4effects_cantbetargeted": 1 if 4 < len(p1.hand) and (p1.hand[4].cant_be_targeted_by_abilities or p1.hand[4].cant_be_targeted_by_hero_powers) else 0,
"myhand4effects_aura": 1 if 4 < len(p1.hand) and p1.hand[4].aura else 0,
"myhand4effects_deathrattle": 1 if 4 < len(p1.hand) and p1.hand[4].has_deathrattle else 0,
"myhand4effects_frozen": 1 if 4 < len(p1.hand) and p1.hand[4].frozen else 0,
"myhand4effects_silenced": 1 if 4 < len(p1.hand) and p1.hand[4].silenced else 0,
"myhand5effects_windfury": 1 if 5 < len(p1.hand) and p1.hand[5].windfury else 0,
"myhand5effects_divineshield": 1 if 5 < len(p1.hand) and p1.hand[5].divine_shield else 0,
"myhand5effects_charge": 1 if 5 < len(p1.hand) and p1.hand[5].charge else 0,
"myhand5effects_taunt": 1 if 5 < len(p1.hand) and p1.hand[5].taunt else 0,
"myhand5effects_stealth": 1 if 5 < len(p1.hand) and p1.hand[5].stealthed else 0,
"myhand5effects_poisonous": 1 if 5 < len(p1.hand) and p1.hand[5].poisonous else 0,
"myhand5effects_cantbetargeted": 1 if 5 < len(p1.hand) and (p1.hand[5].cant_be_targeted_by_abilities or p1.hand[5].cant_be_targeted_by_hero_powers) else 0,
"myhand5effects_aura": 1 if 5 < len(p1.hand) and p1.hand[5].aura else 0,
"myhand5effects_deathrattle": 1 if 5 < len(p1.hand) and p1.hand[5].has_deathrattle else 0,
"myhand5effects_frozen": 1 if 5 < len(p1.hand) and p1.hand[5].frozen else 0,
"myhand5effects_silenced": 1 if 5 < len(p1.hand) and p1.hand[5].silenced else 0,
"myhand6effects_windfury": 1 if 6 < len(p1.hand) and p1.hand[6].windfury else 0,
"myhand6effects_divineshield": 1 if 6 < len(p1.hand) and p1.hand[6].divine_shield else 0,
"myhand6effects_charge": 1 if 6 < len(p1.hand) and p1.hand[6].charge else 0,
"myhand6effects_taunt": 1 if 6 < len(p1.hand) and p1.hand[6].taunt else 0,
"myhand6effects_stealth": 1 if 6 < len(p1.hand) and p1.hand[6].stealthed else 0,
"myhand6effects_poisonous": 1 if 6 < len(p1.hand) and p1.hand[6].poisonous else 0,
"myhand6effects_cantbetargeted": 1 if 6 < len(p1.hand) and (p1.hand[6].cant_be_targeted_by_abilities or p1.hand[6].cant_be_targeted_by_hero_powers) else 0,
"myhand6effects_aura": 1 if 6 < len(p1.hand) and p1.hand[6].aura else 0,
"myhand6effects_deathrattle": 1 if 6 < len(p1.hand) and p1.hand[6].has_deathrattle else 0,
"myhand6effects_frozen": 1 if 6 < len(p1.hand) and p1.hand[6].frozen else 0,
"myhand6effects_silenced": 1 if 6 < len(p1.hand) and p1.hand[6].silenced else 0,
"myhand7effects_windfury": 1 if 7 < len(p1.hand) and p1.hand[7].windfury else 0,
"myhand7effects_divineshield": 1 if 7 < len(p1.hand) and p1.hand[7].divine_shield else 0,
"myhand7effects_charge": 1 if 7 < len(p1.hand) and p1.hand[7].charge else 0,
"myhand7effects_taunt": 1 if 7 < len(p1.hand) and p1.hand[7].taunt else 0,
"myhand7effects_stealth": 1 if 7 < len(p1.hand) and p1.hand[7].stealthed else 0,
"myhand7effects_poisonous": 1 if 7 < len(p1.hand) and p1.hand[7].poisonous else 0,
"myhand7effects_cantbetargeted": 1 if 7 < len(p1.hand) and (p1.hand[7].cant_be_targeted_by_abilities or p1.hand[7].cant_be_targeted_by_hero_powers) else 0,
"myhand7effects_aura": 1 if 7 < len(p1.hand) and p1.hand[7].aura else 0,
"myhand7effects_deathrattle": 1 if 7 < len(p1.hand) and p1.hand[7].has_deathrattle else 0,
"myhand7effects_frozen": 1 if 7 < len(p1.hand) and p1.hand[7].frozen else 0,
"myhand7effects_silenced": 1 if 7 < len(p1.hand) and p1.hand[7].silenced else 0,
"myhand8effects_windfury": 1 if 8 < len(p1.hand) and p1.hand[8].windfury else 0,
"myhand8effects_divineshield": 1 if 8 < len(p1.hand) and p1.hand[8].divine_shield else 0,
"myhand8effects_charge": 1 if 8 < len(p1.hand) and p1.hand[8].charge else 0,
"myhand8effects_taunt": 1 if 8 < len(p1.hand) and p1.hand[8].taunt else 0,
"myhand8effects_stealth": 1 if 8 < len(p1.hand) and p1.hand[8].stealthed else 0,
"myhand8effects_poisonous": 1 if 8 < len(p1.hand) and p1.hand[8].poisonous else 0,
"myhand8effects_cantbetargeted": 1 if 8 < len(p1.hand) and (p1.hand[8].cant_be_targeted_by_abilities or p1.hand[8].cant_be_targeted_by_hero_powers) else 0,
"myhand8effects_aura": 1 if 8 < len(p1.hand) and p1.hand[8].aura else 0,
"myhand8effects_deathrattle": 1 if 8 < len(p1.hand) and p1.hand[8].has_deathrattle else 0,
"myhand8effects_frozen": 1 if 8 < len(p1.hand) and p1.hand[8].frozen else 0,
"myhand8effects_silenced": 1 if 8 < len(p1.hand) and p1.hand[8].silenced else 0,
"myhand9effects_windfury": 1 if 9 < len(p1.hand) and p1.hand[9].windfury else 0,
"myhand9effects_divineshield": 1 if 9 < len(p1.hand) and p1.hand[9].divine_shield else 0,
"myhand9effects_charge": 1 if 9 < len(p1.hand) and p1.hand[9].charge else 0,
"myhand9effects_taunt": 1 if 9 < len(p1.hand) and p1.hand[9].taunt else 0,
"myhand9effects_stealth": 1 if 9 < len(p1.hand) and p1.hand[9].stealthed else 0,
"myhand9effects_poisonous": 1 if 9 < len(p1.hand) and p1.hand[9].poisonous else 0,
"myhand9effects_cantbetargeted": 1 if 9 < len(p1.hand) and (p1.hand[9].cant_be_targeted_by_abilities or p1.hand[9].cant_be_targeted_by_hero_powers) else 0,
"myhand9effects_aura": 1 if 9 < len(p1.hand) and p1.hand[9].aura else 0,
"myhand9effects_deathrattle": 1 if 9 < len(p1.hand) and p1.hand[9].has_deathrattle else 0,
"myhand9effects_frozen": 1 if 9 < len(p1.hand) and p1.hand[9].frozen else 0,
"myhand9effects_silenced": 1 if 9 < len(p1.hand) and p1.hand[9].silenced else 0,
"myfield0effects_windfury": 1 if 0 < len(p1.field) and p1.field[0].windfury else 0,
"myfield0effects_divineshield": 1 if 0 < len(p1.field) and p1.field[0].divine_shield else 0,
"myfield0effects_charge": 1 if 0 < len(p1.field) and p1.field[0].charge else 0,
"myfield0effects_taunt": 1 if 0 < len(p1.field) and p1.field[0].taunt else 0,
"myfield0effects_stealth": 1 if 0 < len(p1.field) and p1.field[0].stealthed else 0,
"myfield0effects_poisonous": 1 if 0 < len(p1.field) and p1.field[0].poisonous else 0,
"myfield0effects_cantbetargeted": 1 if 0 < len(p1.field) and (p1.field[0].cant_be_targeted_by_abilities or p1.field[0].cant_be_targeted_by_hero_powers) else 0,
"myfield0effects_aura": 1 if 0 < len(p1.field) and p1.field[0].aura else 0,
"myfield0effects_deathrattle": 1 if 0 < len(p1.field) and p1.field[0].has_deathrattle else 0,
"myfield0effects_frozen": 1 if 0 < len(p1.field) and p1.field[0].frozen else 0,
"myfield0effects_silenced": 1 if 0 < len(p1.field) and p1.field[0].silenced else 0,
"myfield1effects_windfury": 1 if 1 < len(p1.field) and p1.field[1].windfury else 0,
"myfield1effects_divineshield": 1 if 1 < len(p1.field) and p1.field[1].divine_shield else 0,
"myfield1effects_charge": 1 if 1 < len(p1.field) and p1.field[1].charge else 0,
"myfield1effects_taunt": 1 if 1 < len(p1.field) and p1.field[1].taunt else 0,
"myfield1effects_stealth": 1 if 1 < len(p1.field) and p1.field[1].stealthed else 0,
"myfield1effects_poisonous": 1 if 1 < len(p1.field) and p1.field[1].poisonous else 0,
"myfield1effects_cantbetargeted": 1 if 1 < len(p1.field) and (p1.field[1].cant_be_targeted_by_abilities or p1.field[1].cant_be_targeted_by_hero_powers) else 0,
"myfield1effects_aura": 1 if 1 < len(p1.field) and p1.field[1].aura else 0,
"myfield1effects_deathrattle": 1 if 1 < len(p1.field) and p1.field[1].has_deathrattle else 0,
"myfield1effects_frozen": 1 if 1 < len(p1.field) and p1.field[1].frozen else 0,
"myfield1effects_silenced": 1 if 1 < len(p1.field) and p1.field[1].silenced else 0,
"myfield2effects_windfury": 1 if 2 < len(p1.field) and p1.field[2].windfury else 0,
"myfield2effects_divineshield": 1 if 2 < len(p1.field) and p1.field[2].divine_shield else 0,
"myfield2effects_charge": 1 if 2 < len(p1.field) and p1.field[2].charge else 0,
"myfield2effects_taunt": 1 if 2 < len(p1.field) and p1.field[2].taunt else 0,
"myfield2effects_stealth": 1 if 2 < len(p1.field) and p1.field[2].stealthed else 0,
"myfield2effects_poisonous": 1 if 2 < len(p1.field) and p1.field[2].poisonous else 0,
"myfield2effects_cantbetargeted": 1 if 2 < len(p1.field) and (p1.field[2].cant_be_targeted_by_abilities or p1.field[2].cant_be_targeted_by_hero_powers) else 0,
"myfield2effects_aura": 1 if 2 < len(p1.field) and p1.field[2].aura else 0,
"myfield2effects_deathrattle": 1 if 2 < len(p1.field) and p1.field[2].has_deathrattle else 0,
"myfield2effects_frozen": 1 if 2 < len(p1.field) and p1.field[2].frozen else 0,
"myfield2effects_silenced": 1 if 2 < len(p1.field) and p1.field[2].silenced else 0,
"myfield3effects_windfury": 1 if 3 < len(p1.field) and p1.field[3].windfury else 0,
"myfield3effects_divineshield": 1 if 3 < len(p1.field) and p1.field[3].divine_shield else 0,
"myfield3effects_charge": 1 if 3 < len(p1.field) and p1.field[3].charge else 0,
"myfield3effects_taunt": 1 if 3 < len(p1.field) and p1.field[3].taunt else 0,
"myfield3effects_stealth": 1 if 3 < len(p1.field) and p1.field[3].stealthed else 0,
"myfield3effects_poisonous": 1 if 3 < len(p1.field) and p1.field[3].poisonous else 0,
"myfield3effects_cantbetargeted": 1 if 3 < len(p1.field) and (p1.field[3].cant_be_targeted_by_abilities or p1.field[3].cant_be_targeted_by_hero_powers) else 0,
"myfield3effects_aura": 1 if 3 < len(p1.field) and p1.field[3].aura else 0,
"myfield3effects_deathrattle": 1 if 3 < len(p1.field) and p1.field[3].has_deathrattle else 0,
"myfield3effects_frozen": 1 if 3 < len(p1.field) and p1.field[3].frozen else 0,
"myfield3effects_silenced": 1 if 3 < len(p1.field) and p1.field[3].silenced else 0,
"myfield4effects_windfury": 1 if 4 < len(p1.field) and p1.field[4].windfury else 0,
"myfield4effects_divineshield": 1 if 4 < len(p1.field) and p1.field[4].divine_shield else 0,
"myfield4effects_charge": 1 if 4 < len(p1.field) and p1.field[4].charge else 0,
"myfield4effects_taunt": 1 if 4 < len(p1.field) and p1.field[4].taunt else 0,
"myfield4effects_stealth": 1 if 4 < len(p1.field) and p1.field[4].stealthed else 0,
"myfield4effects_poisonous": 1 if 4 < len(p1.field) and p1.field[4].poisonous else 0,
"myfield4effects_cantbetargeted": 1 if 4 < len(p1.field) and (p1.field[4].cant_be_targeted_by_abilities or p1.field[4].cant_be_targeted_by_hero_powers) else 0,
"myfield4effects_aura": 1 if 4 < len(p1.field) and p1.field[4].aura else 0,
"myfield4effects_deathrattle": 1 if 4 < len(p1.field) and p1.field[4].has_deathrattle else 0,
"myfield4effects_frozen": 1 if 4 < len(p1.field) and p1.field[4].frozen else 0,
"myfield4effects_silenced": 1 if 4 < len(p1.field) and p1.field[4].silenced else 0,
"myfield5effects_windfury": 1 if 5 < len(p1.field) and p1.field[5].windfury else 0,
"myfield5effects_divineshield": 1 if 5 < len(p1.field) and p1.field[5].divine_shield else 0,
"myfield5effects_charge": 1 if 5 < len(p1.field) and p1.field[5].charge else 0,
"myfield5effects_taunt": 1 if 5 < len(p1.field) and p1.field[5].taunt else 0,
"myfield5effects_stealth": 1 if 5 < len(p1.field) and p1.field[5].stealthed else 0,
"myfield5effects_poisonous": 1 if 5 < len(p1.field) and p1.field[5].poisonous else 0,
"myfield5effects_cantbetargeted": 1 if 5 < len(p1.field) and (p1.field[5].cant_be_targeted_by_abilities or p1.field[5].cant_be_targeted_by_hero_powers) else 0,
"myfield5effects_aura": 1 if 5 < len(p1.field) and p1.field[5].aura else 0,
"myfield5effects_deathrattle": 1 if 5 < len(p1.field) and p1.field[5].has_deathrattle else 0,
"myfield5effects_frozen": 1 if 5 < len(p1.field) and p1.field[5].frozen else 0,
"myfield5effects_silenced": 1 if 5 < len(p1.field) and p1.field[5].silenced else 0,
"myfield6effects_windfury": 1 if 6 < len(p1.field) and p1.field[6].windfury else 0,
"myfield6effects_divineshield": 1 if 6 < len(p1.field) and p1.field[6].divine_shield else 0,
"myfield6effects_charge": 1 if 6 < len(p1.field) and p1.field[6].charge else 0,
"myfield6effects_taunt": 1 if 6 < len(p1.field) and p1.field[6].taunt else 0,
"myfield6effects_stealth": 1 if 6 < len(p1.field) and p1.field[6].stealthed else 0,
"myfield6effects_poisonous": 1 if 6 < len(p1.field) and p1.field[6].poisonous else 0,
"myfield6effects_cantbetargeted": 1 if 6 < len(p1.field) and (p1.field[6].cant_be_targeted_by_abilities or p1.field[6].cant_be_targeted_by_hero_powers) else 0,
"myfield6effects_aura": 1 if 6 < len(p1.field) and p1.field[6].aura else 0,
"myfield6effects_deathrattle": 1 if 6 < len(p1.field) and p1.field[6].has_deathrattle else 0,
"myfield6effects_frozen": 1 if 6 < len(p1.field) and p1.field[6].frozen else 0,
"myfield6effects_silenced": 1 if 6 < len(p1.field) and p1.field[6].silenced else 0,
"oppfield0effects_windfury": 1 if 0 < len(p2.field) and p2.field[0].windfury else 0,
"oppfield0effects_divineshield": 1 if 0 < len(p2.field) and p2.field[0].divine_shield else 0,
"oppfield0effects_charge": 1 if 0 < len(p2.field) and p2.field[0].charge else 0,
"oppfield0effects_taunt": 1 if 0 < len(p2.field) and p2.field[0].taunt else 0,
"oppfield0effects_stealth": 1 if 0 < len(p2.field) and p2.field[0].stealthed else 0,
"oppfield0effects_poisonous": 1 if 0 < len(p2.field) and p2.field[0].poisonous else 0,
"oppfield0effects_cantbetargeted": 1 if 0 < len(p2.field) and (p2.field[0].cant_be_targeted_by_abilities or p2.field[0].cant_be_targeted_by_hero_powers) else 0,
"oppfield0effects_aura": 1 if 0 < len(p2.field) and p2.field[0].aura else 0,
"oppfield0effects_deathrattle": 1 if 0 < len(p2.field) and p2.field[0].has_deathrattle else 0,
"oppfield0effects_frozen": 1 if 0 < len(p2.field) and p2.field[0].frozen else 0,
"oppfield0effects_silenced": 1 if 0 < len(p2.field) and p2.field[0].silenced else 0,
"oppfield1effects_windfury": 1 if 1 < len(p2.field) and p2.field[1].windfury else 0,
"oppfield1effects_divineshield": 1 if 1 < len(p2.field) and p2.field[1].divine_shield else 0,
"oppfield1effects_charge": 1 if 1 < len(p2.field) and p2.field[1].charge else 0,
"oppfield1effects_taunt": 1 if 1 < len(p2.field) and p2.field[1].taunt else 0,
"oppfield1effects_stealth": 1 if 1 < len(p2.field) and p2.field[1].stealthed else 0,
"oppfield1effects_poisonous": 1 if 1 < len(p2.field) and p2.field[1].poisonous else 0,
"oppfield1effects_cantbetargeted": 1 if 1 < len(p2.field) and (p2.field[1].cant_be_targeted_by_abilities or p2.field[1].cant_be_targeted_by_hero_powers) else 0,
"oppfield1effects_aura": 1 if 1 < len(p2.field) and p2.field[1].aura else 0,
"oppfield1effects_deathrattle": 1 if 1 < len(p2.field) and p2.field[1].has_deathrattle else 0,
"oppfield1effects_frozen": 1 if 1 < len(p2.field) and p2.field[1].frozen else 0,
"oppfield1effects_silenced": 1 if 1 < len(p2.field) and p2.field[1].silenced else 0,
"oppfield2effects_windfury": 1 if 2 < len(p2.field) and p2.field[2].windfury else 0,
"oppfield2effects_divineshield": 1 if 2 < len(p2.field) and p2.field[2].divine_shield else 0,
"oppfield2effects_charge": 1 if 2 < len(p2.field) and p2.field[2].charge else 0,
"oppfield2effects_taunt": 1 if 2 < len(p2.field) and p2.field[2].taunt else 0,
"oppfield2effects_stealth": 1 if 2 < len(p2.field) and p2.field[2].stealthed else 0,
"oppfield2effects_poisonous": 1 if 2 < len(p2.field) and p2.field[2].poisonous else 0,
"oppfield2effects_cantbetargeted": 1 if 2 < len(p2.field) and (p2.field[2].cant_be_targeted_by_abilities or p2.field[2].cant_be_targeted_by_hero_powers) else 0,
"oppfield2effects_aura": 1 if 2 < len(p2.field) and p2.field[2].aura else 0,
"oppfield2effects_deathrattle": 1 if 2 < len(p2.field) and p2.field[2].has_deathrattle else 0,
"oppfield2effects_frozen": 1 if 2 < len(p2.field) and p2.field[2].frozen else 0,
"oppfield2effects_silenced": 1 if 2 < len(p2.field) and p2.field[2].silenced else 0,
"oppfield3effects_windfury": 1 if 3 < len(p2.field) and p2.field[3].windfury else 0,
"oppfield3effects_divineshield": 1 if 3 < len(p2.field) and p2.field[3].divine_shield else 0,
"oppfield3effects_charge": 1 if 3 < len(p2.field) and p2.field[3].charge else 0,
"oppfield3effects_taunt": 1 if 3 < len(p2.field) and p2.field[3].taunt else 0,
"oppfield3effects_stealth": 1 if 3 < len(p2.field) and p2.field[3].stealthed else 0,
"oppfield3effects_poisonous": 1 if 3 < len(p2.field) and p2.field[3].poisonous else 0,
"oppfield3effects_cantbetargeted": 1 if 3 < len(p2.field) and (p2.field[3].cant_be_targeted_by_abilities or p2.field[3].cant_be_targeted_by_hero_powers) else 0,
"oppfield3effects_aura": 1 if 3 < len(p2.field) and p2.field[3].aura else 0,
"oppfield3effects_deathrattle": 1 if 3 < len(p2.field) and p2.field[3].has_deathrattle else 0,
"oppfield3effects_frozen": 1 if 3 < len(p2.field) and p2.field[3].frozen else 0,
"oppfield3effects_silenced": 1 if 3 < len(p2.field) and p2.field[3].silenced else 0,
"oppfield4effects_windfury": 1 if 4 < len(p2.field) and p2.field[4].windfury else 0,
"oppfield4effects_divineshield": 1 if 4 < len(p2.field) and p2.field[4].divine_shield else 0,
"oppfield4effects_charge": 1 if 4 < len(p2.field) and p2.field[4].charge else 0,
"oppfield4effects_taunt": 1 if 4 < len(p2.field) and p2.field[4].taunt else 0,
"oppfield4effects_stealth": 1 if 4 < len(p2.field) and p2.field[4].stealthed else 0,
"oppfield4effects_poisonous": 1 if 4 < len(p2.field) and p2.field[4].poisonous else 0,
"oppfield4effects_cantbetargeted": 1 if 4 < len(p2.field) and (p2.field[4].cant_be_targeted_by_abilities or p2.field[4].cant_be_targeted_by_hero_powers) else 0,
"oppfield4effects_aura": 1 if 4 < len(p2.field) and p2.field[4].aura else 0,
"oppfield4effects_deathrattle": 1 if 4 < len(p2.field) and p2.field[4].has_deathrattle else 0,
"oppfield4effects_frozen": 1 if 4 < len(p2.field) and p2.field[4].frozen else 0,
"oppfield4effects_silenced": 1 if 4 < len(p2.field) and p2.field[4].silenced else 0,
"oppfield5effects_windfury": 1 if 5 < len(p2.field) and p2.field[5].windfury else 0,
"oppfield5effects_divineshield": 1 if 5 < len(p2.field) and p2.field[5].divine_shield else 0,
"oppfield5effects_charge": 1 if 5 < len(p2.field) and p2.field[5].charge else 0,
"oppfield5effects_taunt": 1 if 5 < len(p2.field) and p2.field[5].taunt else 0,
"oppfield5effects_stealth": 1 if 5 < len(p2.field) and p2.field[5].stealthed else 0,
"oppfield5effects_poisonous": 1 if 5 < len(p2.field) and p2.field[5].poisonous else 0,
"oppfield5effects_cantbetargeted": 1 if 5 < len(p2.field) and (p2.field[5].cant_be_targeted_by_abilities or p2.field[5].cant_be_targeted_by_hero_powers) else 0,
"oppfield5effects_aura": 1 if 5 < len(p2.field) and p2.field[5].aura else 0,
"oppfield5effects_deathrattle": 1 if 5 < len(p2.field) and p2.field[5].has_deathrattle else 0,
"oppfield5effects_frozen": 1 if 5 < len(p2.field) and p2.field[5].frozen else 0,
"oppfield5effects_silenced": 1 if 5 < len(p2.field) and p2.field[5].silenced else 0,
"oppfield6effects_windfury": 1 if 6 < len(p2.field) and p2.field[6].windfury else 0,
"oppfield6effects_divineshield": 1 if 6 < len(p2.field) and p2.field[6].divine_shield else 0,
"oppfield6effects_charge": 1 if 6 < len(p2.field) and p2.field[6].charge else 0,
"oppfield6effects_taunt": 1 if 6 < len(p2.field) and p2.field[6].taunt else 0,
"oppfield6effects_stealth": 1 if 6 < len(p2.field) and p2.field[6].stealthed else 0,
"oppfield6effects_poisonous": 1 if 6 < len(p2.field) and p2.field[6].poisonous else 0,
"oppfield6effects_cantbetargeted": 1 if 6 < len(p2.field) and (p2.field[6].cant_be_targeted_by_abilities or p2.field[6].cant_be_targeted_by_hero_powers) else 0,
"oppfield6effects_aura": 1 if 6 < len(p2.field) and p2.field[6].aura else 0,
"oppfield6effects_deathrattle": 1 if 6 < len(p2.field) and p2.field[6].has_deathrattle else 0,
"oppfield6effects_frozen": 1 if 6 < len(p2.field) and p2.field[6].frozen else 0,
"oppfield6effects_silenced": 1 if 6 < len(p2.field) and p2.field[6].silenced else 0,"""

Este es el código para añadir la condición de que no sea una carta de tipo SPELL

In [257]:
modify=["divineshield", "charge", "taunt", "stealth", "poisonous", "cantbetargeted", "deathrattle", "frozen", "silenced"]
line='"oppfield3effects_frozen": 1 if 3 < len(p2.field) and p2.field[3].frozen else 0,'
add_string="{}.{}[{}].type!=5"
p="p1" if "p1" in line else "p2"
fildhand="field" if "field" in line else "hand"
position=re.search(r'\[(\d+)\]', line).group(1)
components=line.split(" and ")
components.insert(1,add_string.format(p,fildhand,position))
newline=" and ".join(components)
newline

'"oppfield3effects_frozen": 1 if 3 < len(p2.field) and p2.field[3].type!=5 and p2.field[3].frozen else 0,'

Y este el código que lo aplica:

In [258]:
newcode=[]
modify=["divineshield", "charge", "taunt", "stealth", "poisonous", "cantbetargeted", "deathrattle", "frozen", "silenced"]
add_string="{}.{}[{}].type!=5"
for line in code.split("\n"):
    found=False
    for word in modify:
        if word in line:
            found=True
            p="p1" if "p1" in line else "p2"
            fildhand="field" if "field" in line else "hand"
            position=re.search(r'\[(\d+)\]', line).group(1)
            components=line.split(" and ")
            if len(components)!=2:
                print("ERROR")
            else:
                components.insert(1,add_string.format(p,fildhand,position))
                newline=" and ".join(components)
                newcode.append(newline)
                break
    if found==False:
        newcode.append(line) 

In [259]:
len(newcode)==len(code.split("\n"))

True

Este es el resultado que nos llevamos al hearthstone_unnested_env.py

In [260]:
print("\n".join(newcode))

""myhand0effects_windfury": 1 if 0 < len(p1.hand) and p1.hand[0].windfury else 0,
"myhand0effects_divineshield": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and p1.hand[0].divine_shield else 0,
"myhand0effects_charge": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and p1.hand[0].charge else 0,
"myhand0effects_taunt": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and p1.hand[0].taunt else 0,
"myhand0effects_stealth": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and p1.hand[0].stealthed else 0,
"myhand0effects_poisonous": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and p1.hand[0].poisonous else 0,
"myhand0effects_cantbetargeted": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and (p1.hand[0].cant_be_targeted_by_abilities or p1.hand[0].cant_be_targeted_by_hero_powers) else 0,
"myhand0effects_aura": 1 if 0 < len(p1.hand) and p1.hand[0].aura else 0,
"myhand0effects_deathrattle": 1 if 0 < len(p1.hand) and p1.hand[0].type!=5 and p1.hand[0].has_deathrattle else 0,
"myhand0effects_frozen": 1 if 0

In [36]:
from hearthstone.enums import PlayState, Step, Mulligan, State, CardClass, Race, CardSet, CardType

list(CardClass)

[<CardClass.INVALID: 0>,
 <CardClass.DEATHKNIGHT: 1>,
 <CardClass.DRUID: 2>,
 <CardClass.HUNTER: 3>,
 <CardClass.MAGE: 4>,
 <CardClass.PALADIN: 5>,
 <CardClass.PRIEST: 6>,
 <CardClass.ROGUE: 7>,
 <CardClass.SHAMAN: 8>,
 <CardClass.WARLOCK: 9>,
 <CardClass.WARRIOR: 10>,
 <CardClass.DREAM: 11>,
 <CardClass.NEUTRAL: 12>,
 <CardClass.WHIZBANG: 13>,
 <CardClass.DEMONHUNTER: 14>]

In [68]:
class HearthstoneUnnestedEnv():
    
    def setup_game(self):
            while True:
                print("iter")
                try:
                    # At the root pretend the player just moved is p2 - p1 has the first move
                    self.playerJustMoved=2
                    self.playerToMove=1
                    self.players_ordered=None
                    self.hero1=random.choice(list(CardClass))
                    self.deck1=None
                    self.hero2=random.choice(list(CardClass))
                    self.deck2=None
                    self.game=None
                    self.lastMovePlayed=None
                    if self.hero1 is None or self.hero2 is None or self.deck1 is None or self.deck2 is None:
                        self.deck1=[]
                        self.deck2=[]
                        self.collection1 = []
                        self.collection2 = []
                        for card in cards.db.keys():
                            if str(card) not in implemented_cards:
                                continue
                            cls = cards.db[card]
                            if not cls.collectible:
                                continue
                            if cls.type == CardType.HERO:
                                # Heroes are collectible...
                                continue
                            if cls.card_class and cls.card_class in [self.hero1, CardClass.NEUTRAL]:
                                # Play with more possibilities
                                self.collection1.append(cls)
                            if cls.card_class and cls.card_class in [self.hero2, CardClass.NEUTRAL]:
                                # Play with more possibilities
                                self.collection2.append(cls)
                            

                        while len(self.deck1) < Deck.MAX_CARDS:
                            card = random.choice(self.collection1)
                            if self.deck1.count(card.id) < card.max_count_in_deck:
                                self.deck1.append(card.id)

                        while len(self.deck2) < Deck.MAX_CARDS:
                            card = random.choice(self.collection2)
                            if self.deck2.count(card.id) < card.max_count_in_deck:
                                self.deck2.append(card.id)

                        self.player1=Player("Player1", self.deck1, self.hero1.default_hero)
                        self.player2=Player("Player2", self.deck2, self.hero2.default_hero)

                        self.game=Game(players=(self.player1, self.player2))
                        self.game.start()
                        self.players_ordered=[self.game.player1, self.game.player2]
                        # At the root pretend the player just moved is p2 - p1 has the first move
                        self.playerJustMoved=2
                        self.playerToMove=1
                        self.lastMovePlayed=None
                        return
                except Exception as ex:
                    print("exception making decks--trying again"+str(ex))

In [69]:
a=HearthstoneUnnestedEnv()
a.setup_game()

iter


[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player1 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Steady Shot')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Steady Shot')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Rexxar')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Rexxar')>)> targeting [Player(name='Player1', hero=<Hero ('Rexxar')>)]
[fireplace.actions]: Player1 summons [<Hero ('Rexxar')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Rexxar')>) shuffles their deck
[fireplace.card]: <Minion ('Cauldron Elemental')> moves from <Zone.D

In [70]:
print(len(a.collection1))
print(len(a.collection2))

872
859


In [77]:
a.player1.choice.cards

[<Minion ('Cauldron Elemental')>,
 <Spell ('Cybertech Chip')>,
 <Minion ("Y'Shaarj, Rage Unbound")>]

In [78]:
a.player1.hand

[<Minion ('Cauldron Elemental')>,
 <Spell ('Cybertech Chip')>,
 <Minion ("Y'Shaarj, Rage Unbound")>]

In [79]:
a.game.current_player

**\_\_init\_\_**  Inicializa los espacios de acción y observaciones y llama a **setup_game()**

**reset**  Llama a **setup_game()** y devuelve el estado inicial: **self._get_state()** definida en **gym.Env**

**setup_game()** Realiza varias acciones:
* Elige un HERO para cada jugador
* Crea un deck para cada jugador
* Inicializa el juego e invoca a la funcion START()

**step(action)** El método se limita a invocar a las funciones **_take_action(action)**, **._get_reward()** y **_get_state()**. Se le pasa una de las 869 acciones definidas y devuelve 4 elementos:
* una observación del entorno
* la reward conseguida con esa acción
* buleano diciendo si el episodio a terminado
* info para debugging  

**\_\_getMoves**: Pedirle a Jesús que lo revise. En función del momento del juego, los movimientos posibles son Move.mulligan, Move.choice o Move.play_card, Move.hero_power, Move.minion_attack, Move.hero_attack y Move.end_turn. Ojo, el movimiento Move.end_turn parece que no se añade siempre. Y cuando se meter por el ELSE identificado como "# Play card", valid_moves no sería una lista vacía?? Por qué hace un range(len(valid_moves))???


**\_\_doMove**: Pedirle a Jesús que lo revise. Entre otras cosas, cambia el valor del turno del jugador. 


**_take_action** Empieza calculando cuales son las posibles acciones. Y de todas las posibles acciones eliminar acciones que ya hayan sido seleccionadas antes (y que estarán en la lista **alreadySelectedActions**). El caso es que hay un punto de fallo en el que se eliminan las acciones que ya se han elegido y puede darse la circunstancia que las posibles acciones queden a 0. Por lo que el programa falla. Preguntar a Jesús si tiene sentido.
Quiero entender que un jugador puede hacer todos los movimientos que quiera antes de pasar el turno?? La función **\_\_getMoves** debería de devolver una lista con acciones entre las que se incluya el valor Move.end_turn??

El funcionamiento en general es que la AI que estamos entrenado realiza la acción pedida y la añade a la lista **alreadySelectedActions** para evitar que se vuelva a usar otra vez. En caso de que la acción seleccionada fuera una pérdida de turno para la AI, entonces:
1. se le pasa el turno al otro player (llamado RANDOM) invocando a la función \_\_doMove con la acción
2. se vacía la lista alreadySelectedActions 
3. se obtienen las posibles acciones del RANDOM player
4. se elige una acción random de entre las disponibles
5. se añade a la lista alreadySelectedActions
6. se realiza la acción elegida
7. Se vuelven a calcular las posibles acciones
8. se eliminan las que ya se hayan elegido antes
9. se vuelve a elegir otra random. 
10. Y así hasta que se sevuelva la acción Move.end_turn en cuyo caso se ejecuta, se vacía la lista alreadySelectedActions y se le pasa el turno al jugador AI


Tampoco entiendo porqué si hay 869 acciones que parece que están asignadas, al final, se llena la lista de 869 acciones con repeticiones de las acciones posibles. No debería corresponderse siempre un número con la misma acción????

**\_get_reward**  

In [34]:
#!/usr/bin/env python
import argparse
import re
import string
import sys

from hearthstone.enums import CardSet

from fireplace import cards
from fireplace.utils import get_script_definition



GREEN = "\033[92m"
RED = "\033[91m"
ENDC = "\033[0m"
PREFIXES = {
    GREEN: "Implemented",
    RED: "Not implemented",
}

SOLVED_KEYWORDS = [
    "Windfury", "Charge", "Divine Shield", "Taunt", "Stealth", "Poisonous",
    r"Can't be targeted by spells or Hero Powers\.",
    r"Can't attack\.",
    "Destroy any minion damaged by this minion.",
    r"Your Hero Power deals \d+ extra damage.",
    r"Spell Damage \+\d+",
    r"Overload: \(\d+\)",
]

DUMMY_CARDS = (
    "PlaceholderCard",  # Placeholder Card
    "CS2_022e",  # Polymorph
    "EX1_246e",  # Hexxed
    "EX1_345t",  # Shadow of Nothing
    "GAME_006",  # NOOOOOOOOOOOO
    "LOEA04_27",  # Animated Statue
    "Mekka4e",  # Transformed
    "NEW1_025e",  # Bolstered (Unused)
    "TU4c_005",  # Hidden Gnome
    "TU4c_007",  # Mukla's Big Brother

    # Dynamic buffs set by their parent
    "CS2_236e",  # Divine Spirit
    "EX1_304e",  # Consume (Void Terror)
    "LOE_030e",  # Hollow (Unused)
    "NEW1_018e",  # Treasure Crazed (Bloodsail Raider)
)


def cleanup_description(description):
    ret = description
    ret = re.sub("<i>.+</i>", "", ret)
    ret = re.sub("(<b>|</b>)", "", ret)
    ret = re.sub("(" + "|".join(SOLVED_KEYWORDS) + ")", "", ret)
    ret = re.sub("<[^>]*>", "", ret)
    exclude_chars = string.punctuation + string.whitespace
    ret = "".join([ch for ch in ret if ch not in exclude_chars])
    return ret


def implemented():
    impl = 0
    unimpl = 0

#     p = argparse.ArgumentParser()
#     p.add_argument(
#         "--implemented",
#         action="store_true",
#         dest="implemented",
#         help="Show only implemented cards"
#     )
#     p.add_argument(
#         "--unimplemented",
#         action="store_true",
#         dest="unimplemented",
#         help="Show only unimplemented cards"
#     )
#     args = p.parse_args(sys.argv[1:])

#     if not args.implemented and not args.unimplemented:
#         args.implemented = True
#         args.unimplemented = True

    cards.db.initialize()
    for id in sorted(cards.db):
        card = cards.db[id]
        description = cleanup_description(card.description)
        implemented = False

        if not description:
            # Minions without card text or with basic abilities are implemented
            implemented = True
        elif card.card_set == CardSet.CREDITS:
            implemented = True

        if id in DUMMY_CARDS:
            implemented = True

        carddef = get_script_definition(id)
        if carddef:
            implemented = True

        color = GREEN if implemented else RED
        name = color + "%s: %s" % (PREFIXES[color], card.name) + ENDC

        if implemented:
            impl += 1
#             if args.implemented:
#                 print("%s (%s)" % (name, id))
        else:
            unimpl += 1
#             if args.unimplemented:
#                 print("%s (%s)" % (name, id))

    total = impl + unimpl

    print("%i / %i cards implemented (%i%%)" % (impl, total, (impl / total) * 100))

In [35]:
implemented()

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 25793 cards


7923 / 25793 cards implemented (30%)
